### _`Creating a  Rest API for an existing AWS Sagemaker Endpoint by using AWS Lambda & AWS API Gateway`_

#### _`Let's first of all, create our lambda function (a serverless AWS service) that will preprocess the incoming requests on our behave and invoke the corresponding AWS SageMaker Endpoint`_

In [ ]:
import os
import io
import boto3
import json
import csv

# For best practice reasons, we will fetch the sagemaker endpoint from the environment variables
ENDPOINT_NAME = os.environ['ENDPOINT_NAME']

# And then we will initialize the runtime client for invoking the SageMaker endpoint
runtime= boto3.client('sagemaker-runtime') # 'runtime.sagemaker'

# Now, comes the incoming request processing part
def lambda_handler(event, context):
    # Let's first of all store all the event JSON format into a variable 
    payload = json.loads(json.dumps(event))

    # Then, we will extract the data from the request body (which is a CSV string)
    payload_data = str(payload['body'])

    # Then, we will try to print the extracted data to ensure it's being processed correctly
    print(payload_data)

    # Now, it is high time to invoke the endpoint 
    response = runtime.invoke_endpoint(EndpointName=ENDPOINT_NAME,
                                      ContentType='text/csv',
                                      Body=payload_data)
    
    # After invoking the endpoint, we will parse the response to get the prediction
    result = json.loads(response['Body'].read().decode())

    # And finally, we will send the response back to the client
    preds = {"Prediction": result}
    
    response_dict = {
          "statusCode": 200,
          "body": json.dumps(preds)
                }
    return response_dict

### _`Now, comes one of the most important parts, giving the permissions to the lambda function to be able to invoke the SageMaker endpoint`_

In [ ]:
{
    "Version": "2012-10-17",
    "Statement": [
        {
            "Sid": "VisualEditor0",
            "Effect": "Allow", # We are allowing actions/operations that will be specified in 'Action'
            "Action": "sagemaker:InvokeEndpoint", # The action/operation that we are allowing is invoking a sagemaker endpoint 
            "Resource": "*" # On all the sagemaker's endpoints (here, we are not specifying endpoints, but all of them)
        }
    ]
}

### _`Now that our lambda function is ready, we will now create our  Rest API in API Gateway and configure it to send the incoming requests' events to the above lambda function for prediction`_

_`Now that our REST API is created and configured to send the incoming requests' event to our desired lambda function, now it's time to test our 'predict' resource and our 'POST' method `_

In [ ]:
import requests

api_url = " <THE API URL>" 

payload = "1454.0,1.0,0.5,1.0,1.0,0.0,34.0,0.7,83.0,4.0,3.0,250.0,1033.0,3419.0,7.0,5.0,5.0,1.0,1.0,0.0"

headers = {
  'Content-Type': 'text/csv' # Because, we have prepared the model to receive the incoming request data as a 'text/csv' in our 'def input_fn' function (you can find it in 
                             # the inference.py) python script in the same directory
}

response = requests.request("POST", api_url, headers=headers, data=payload)

print(response.text)

### _`After testing the API, if we are satisfied with it, we can directly deploy it in a very simple way from the AWS API Gateway`_

## _`Congratulations!!! You have built, trained, and deployed your own custom models and scripts in AWS SageMaker from a business problem to the deployment to the end users in AWS Cloud.`_